In [1]:
import pandas as pd;
import numpy as np;
import matplotlib.pyplot as plt;
import seaborn as sns;
import os;
import time;
from PIL import Image;
from tqdm import tqdm;
import shutil;
%matplotlib inline

In [187]:
!dir

 Volume in drive C has no label.
 Volume Serial Number is 2EBD-2BDC

 Directory of C:\Users\Shahbaz\Downloads\GuessTheArt\GuessTheRealArt

07/11/2023  01:44 PM    <DIR>          .
07/11/2023  01:44 PM    <DIR>          ..
07/09/2023  08:56 AM                68 .gitattributes
07/09/2023  08:56 AM             1,389 .gitignore
07/11/2023  01:20 PM    <DIR>          .ipynb_checkpoints
07/11/2023  01:16 PM    <DIR>          Assets_to_import
07/09/2023  08:56 AM    <DIR>          examples
07/11/2023  01:44 PM    <DIR>          famous-art-512-sq
07/11/2023  12:44 PM    <DIR>          famous-art-512-sq_Preserved
07/09/2023  08:56 AM    <DIR>          famous-paintings-by-author
07/10/2023  01:21 PM    <DIR>          genned_images
07/09/2023  08:56 AM    <DIR>          GuessTheRealArt
07/09/2023  08:56 AM    <DIR>          images
07/11/2023  01:44 PM            41,788 JSON_Automater.ipynb
07/10/2023  07:17 PM    <DIR>          landscapes
07/11/2023  01:43 PM            19,900 original_512_name_f

## Set Real Image Parameteres and Image Set Paths

In [2]:
# this is the pre-cropped 512x512 version of the images in the dataset
answer_path = './famous-art-512-sq/';
# path to the folder containing the generated images
genned_images = './processed/';
# path to re-create file structure for image set for importing to assets directory
to_export = './Assets_to_import/'
# path containing original file names before 'aaaaa_' insertion
of_names_directory = './famous-art-512-sq_Preserved/'

## DataFrames with the list of original and generated images

In [3]:
# original image file data
df_originals = pd.DataFrame(os.listdir(answer_path))
df_originals.columns = ['file_names']
df_originals.sort_values(by = 'file_names', inplace = True)
df_originals['file_names'][16]

'Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S-2.jpg'

In [4]:
# Generated image file data
df_genned = pd.DataFrame(os.listdir(genned_images))
df_genned.columns = ['file_names']
df_genned.sort_values(by = 'file_names', inplace = True)
df_genned.head()

,file_names
0,Adolph-Von-Menzel-The-Flute-Concert-at-Sanssou...
1,Adolph-Von-Menzel-The-Flute-Concert-at-Sanssou...
2,Adolph-Von-Menzel-The-Flute-Concert-at-Sanssou...
4,Adolph-Von-Menzel-The-Flute-Concert-at-Sanssou...
5,Adolph-Von-Menzel-The-Flute-Concert-at-Sanssou...


In [5]:
print ('Number of original images: ', len(df_originals), '\n')
print ('Number of genned images: ', len(df_genned), '\n')

Number of original images:  924 

Number of genned images:  1589 



In [6]:
df_originals['file_names'][16]

'Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S-2.jpg'

## Naming convention and breakdown

The generated images have the filenames in the following format: <br>
\<original_name> + \<[underscore]> + \<concept_applied> + \<'[underscore]'> + \<guidance scale value> + \<file_extension>

## Creating a folder for each original artwork that has generated images

In [64]:
sanitized_genned = df_genned['file_names'].apply(lambda x: x.split('_')[0])

# number of images in 'complete set'
n_images_complete_set = 3

In [65]:
print ('Number of originals which have concepts generated: \t\t\t', len(sanitized_genned.unique()))

print ('Number of completed sets, with all concepts applied to original image: \t', (sanitized_genned.value_counts() >= n_images_complete_set).sum())

print ('Artwork(s) for which the generation set is incomplete: \t\t\t', sanitized_genned.unique()[(sanitized_genned.value_counts() < n_images_complete_set)][0], '\n')

Number of originals which have concepts generated: 			 25
Number of completed sets, with all concepts applied to original image: 	 25


IndexError: index 0 is out of bounds for axis 0 with size 0

In [66]:
sanitized_genned.value_counts()

Albert-Edelfelt-The-Artist+s-Son-Erik-in-his-Pram-S.s-Son-Erik-in-his-Pram-S    66
Alexandre-Cabanel-Fallen-Angel-S                                                66
Albrecht-Durer-Young-Hare-S                                                     66
Albrecht-Durer-The-Martyrdom-of-the-Ten-Thousand-S                              66
Albrecht-Durer-The-Altarpiece-of-the-Rose-Garlands-S                            66
Albrecht-Durer-Self-portrait-at-22-S                                            66
Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S                             66
Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S+002.002                     66
Albrecht-Durer-Christ-as-the-Man-of-Sorrows-S                                   66
Albrecht-Durer-Christ-Among-the-Doctors-S                                       66
Agnolo-Bronzino-Venus-Cupid-and-Time-Allegory-of-Lust--S                        66
AlbrechtDurer-Selfportraitdetailface-S                                          66
Albe

In [67]:
#pd.DataFrame(sanitized_genned.unique())
[i for i in range(len(sanitized_genned.unique())) if df_originals['file_names'][i][:-6] == sanitized_genned.unique()[i]]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 12,
 13,
 14,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24]

In [59]:
df_originals['genned_set_name'] = df_originals['file_names'].apply(lambda x: '-'.join(x.split('-')[:-1]))
df_originals.sort_values(by = 'genned_set_name')['genned_set_name'][2]
#df_originals['genned_set_name'][19]

'Albert-Bierstadt-Among-the-Sierra-Nevada-Mountains-Californi'

In [58]:
genned_set = pd.DataFrame(sanitized_genned.unique())[0].apply(lambda x: '-'.join(x.split('-')[:]) if (len(x.split('.')) < 2) else '-'.join(x.split('.')[:-1]));
genned_set.sort_values()[2]
#genned_set

#genned_set = pd.DataFrame(sanitized_genned.unique())[0].apply(lambda x: x.split('.'));
#genned_set.sort_values()


'Albert-Bierstadt-Among-the-Sierra-Nevada-Mountains-Californi'

In [60]:
[i for i in range(len(genned_set)) if genned_set[i] == df_originals['genned_set_name'][i]]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24]

In [61]:
pd.DataFrame([genned_set, df_originals['genned_set_name']]).transpose().dropna()

,0,genned_set_name
0,Adolph-Von-Menzel-The-Flute-Concert-at-Sanssou...,Adolph-Von-Menzel-The-Flute-Concert-at-Sanssou...
1,Agnolo-Bronzino-Venus-Cupid-and-Time-Allegory-...,Agnolo-Bronzino-Venus-Cupid-and-Time-Allegory-...
2,Albert-Bierstadt-Among-the-Sierra-Nevada-Mount...,Albert-Bierstadt-Among-the-Sierra-Nevada-Mount...
3,Albert-Bierstadt-Deer-in-a-Clearing-S,Albert-Bierstadt-Deer-in-a-Clearing-S
4,Albert-Bierstadt-Forest-Sunrise-S,Albert-Bierstadt-Forest-Sunrise-S
5,Albert-Bierstadt-Rocky-Mountains-S,Albert-Bierstadt-Rocky-Mountains-S
6,Albert-Bierstadt-Rustic-Mill-S,Albert-Bierstadt-Rustic-Mill-S
7,Albert-Bierstadt-Staubbach-Falls-Near-Lauterbr...,Albert-Bierstadt-Staubbach-Falls-Near-Lauterbr...
8,Albert-Charles-Lebourg-Albert-Marie-Lebourg-Ro...,Albert-Charles-Lebourg-Albert-Marie-Lebourg-Ro...
9,Albert-Charles-Lebourg-Notre-Dame-de-Paris-and...,Albert-Charles-Lebourg-Notre-Dame-de-Paris-and...


In [73]:
df_originals['genned_set_name'][15]

'Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S+002'

In [70]:
genned_set.unique()[15]

'Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S+002'

### Creating a folder for each set which has more than \<n_images_complete_set\> genned files (minimum for a set)

In [74]:
print('folders will be created in the following path: \n' + to_export)

folders will be created in the following path: 
./Assets_to_import/


In [89]:
filepath = '';
for i in tqdm(genned_set):
    filepath = to_export + i
    if not(os.path.exists(filepath)):
        os.mkdir(filepath)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 1927.64it/s]


### Create copies of each original file and append 'aaaaa' to the beginning of filename

In [76]:
print('files will be renamed from the following path: \n' + answer_path)

files will be renamed from the following path: 
./famous-art-512-sq/


In [77]:
### to rename each file and append 'aaaaa' to it
filepath = ''
filename_to = ''
for i in tqdm(df_originals['file_names']):
    
    filepath = answer_path + "/" + i;
    filename_to = answer_path + "/aaaaa_" + i;
    
    os.rename(src = filepath, dst = filename_to)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 924/924 [00:00<00:00, 1203.82it/s]


In [78]:
## reflect changed filenames in another column for the originals dataframe
df_originals['new_file_names'] = os.listdir(answer_path);
df_originals['new_file_names']

0      aaaaa_Adolph-Von-Menzel-The-Flute-Concert-at-S...
1      aaaaa_Agnolo-Bronzino-Venus-Cupid-and-Time-All...
2      aaaaa_Albert-Bierstadt-Among-the-Sierra-Nevada...
3      aaaaa_Albert-Bierstadt-Deer-in-a-Clearing-S-2.jpg
4          aaaaa_Albert-Bierstadt-Forest-Sunrise-S-2.jpg
                             ...                        
919       aaaaa_Yoshitomo-Nara-The-longest-night-S-2.jpg
920    aaaaa_Yves-Tanguy-Indefinite-divisibility-S-2.jpg
921    aaaaa_Yves-Tanguy-Multiplicatiion-of-the-Arcs-...
922    aaaaa_Yves-Tanguy-Slowly-Toward-The-North-S-2.jpg
923       aaaaa_Yves-Tanguy-The-Ribbon-of-Excess-S-2.jpg
Name: new_file_names, Length: 924, dtype: object

In [79]:
# check for integrity data we will use to correlate the sets
df_originals['genned_set_name'][:(len(genned_set))] == genned_set

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
20    True
21    True
22    True
23    True
24    True
dtype: bool

In [80]:
#print (df_originals[df_originals['genned_set_name'][:(len(genned_set))] == genned_set])
print(df_originals[:(len(genned_set))][(df_originals['genned_set_name'][:(len(genned_set))] != genned_set)]['genned_set_name'].values)
print (genned_set[(df_originals['genned_set_name'][:(len(genned_set))] != genned_set)].values)

[]
[]


In [81]:
### fixing incorrect values manually

#df_originals['genned_set_name'][11] = df_originals['genned_set_name'][11][:-2]
#df_originals['genned_set_name'][11]
#genned_set
#df_originals['genned_set_name'][21] = df_originals['genned_set_name'][21][:-6]
#df_originals['genned_set_name'][21]


In [83]:
print (genned_set)

0     Adolph-Von-Menzel-The-Flute-Concert-at-Sanssou...
1     Agnolo-Bronzino-Venus-Cupid-and-Time-Allegory-...
2     Albert-Bierstadt-Among-the-Sierra-Nevada-Mount...
3                 Albert-Bierstadt-Deer-in-a-Clearing-S
4                     Albert-Bierstadt-Forest-Sunrise-S
5                    Albert-Bierstadt-Rocky-Mountains-S
6                        Albert-Bierstadt-Rustic-Mill-S
7     Albert-Bierstadt-Staubbach-Falls-Near-Lauterbr...
8     Albert-Charles-Lebourg-Albert-Marie-Lebourg-Ro...
9     Albert-Charles-Lebourg-Notre-Dame-de-Paris-and...
10    Albert-Charles-Lebourg-The-Quays-of-Dieppe-aft...
11    Albert-Edelfelt-The-Artist+s-Son-Erik-in-his-P...
12    Albrecht-Altdorfer-The-Battle-of-Alexander-at-...
13            Albrecht-Durer-Christ-Among-the-Doctors-S
14        Albrecht-Durer-Christ-as-the-Man-of-Sorrows-S
15    Albrecht-Durer-Portrait-of-a-Young-Venetian-Wo...
16    Albrecht-Durer-Portrait-of-a-Young-Venetian-Wo...
17                 Albrecht-Durer-Self-portrait-

In [90]:
# re-check for integrity data we will use to correlate the sets
df_originals['genned_set_name'][:(len(genned_set))] == genned_set

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
20    True
21    True
22    True
23    True
24    True
dtype: bool

In [91]:
### Names of folders we will be copying each original-renamed file to
pd.Series(os.listdir(to_export))[:len(genned_set)] == genned_set[:len(os.listdir(to_export))]

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15    False
16    False
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
dtype: bool

In [93]:
print(genned_set[15])
print(os.listdir(to_export)[15] + '\n')

print(genned_set[16])
print(os.listdir(to_export)[16] + '\n')

print(genned_set[17])
print(os.listdir(to_export)[17] + '\n')

Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S+002
Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S

Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S
Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S+002

Albrecht-Durer-Self-portrait-at-22-S
Albrecht-Durer-Self-portrait-at-22-S



In [137]:
df_genned['copy_to'] = df_genned['file_names'].apply(lambda x: (x.split('_')[:-2])[0])
print(df_genned['file_names'][66*15])
print(df_genned['copy_to'][66*15], '\n')

df_genned['copy_to'] = df_genned['copy_to'].apply(lambda x: '-'.join(x.split('-')[:]) if (len(x.split('.')) < 2) else '-'.join(x.split('.')[:-1]))
print(df_genned['file_names'][66*15])
print(df_genned['copy_to'][66*15], '\n')


Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S+002.002_blue_8.jpg
Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S+002.002 

Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S+002.002_blue_8.jpg
Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S+002 



In [103]:
### move each file to its corresponding complete set folder generated in the previous subsection

os.listdir()

filepath = ''
#filename_to = ''
#for i in genned_set:
    
    

['Adolph-Von-Menzel-The',
 'Agnolo-Bronzino-Venus-Cupid',
 'Albert-Bierstadt-Among-the',
 'Albert-Bierstadt-Deer-in',
 'Albert-Bierstadt-Forest-Sunrise',
 'Albert-Bierstadt-Rocky-Mountains',
 'Albert-Bierstadt-Rustic-Mill',
 'Albert-Bierstadt-Staubbach-Falls',
 'Albert-Charles-Lebourg-Albert',
 'Albert-Charles-Lebourg-Notre',
 'Albert-Charles-Lebourg-The',
 'Albert-Edelfelt-The-Artist',
 'Albrecht-Altdorfer-The-Battle',
 'Albrecht-Durer-Christ-Among',
 'Albrecht-Durer-Christ-as',
 'Albrecht-Durer-Portrait-of',
 'Albrecht-Durer-Self-portrait',
 'Albrecht-Durer-The-Altarpiece',
 'Albrecht-Durer-The-Martyrdom',
 'Albrecht-Durer-Young-Hare',
 'AlbrechtDurer-Selfportraitdetailface-S',
 'Alexandre-Cabanel-Fallen-Angel',
 'Alexej-Georgewitsch-Von-Jawlensky',
 'Alfred-Sisley-Foggy-Morning']

### Copy all generated files into respective file directory in buffer_asset folder

In [149]:
### checking for integrity of dataframe data
print(df_originals['genned_set_name'][0])
print (df_genned['copy_to'][0 * 66])
print(os.listdir(to_export)[0], '\n')


print(df_originals['genned_set_name'][1])
print (df_genned['copy_to'][1 * 66]) # because the 66th image is the first of the next set for the initial few
print(os.listdir(to_export)[1], '\n')

# for special cases:
print(df_originals['genned_set_name'][15])
print (df_genned['copy_to'][15 * 66]) # because the 66th image is the first of the next set for the initial few
print(os.listdir(to_export)[16] ,'\n')

print(df_originals['genned_set_name'][16])
print (df_genned['copy_to'][16 * 66]) # because the 66th image is the first of the next set for the initial few
print(os.listdir(to_export)[15] ,'\n')



Adolph-Von-Menzel-The-Flute-Concert-at-Sanssouci-sketch--S
Adolph-Von-Menzel-The-Flute-Concert-at-Sanssouci-sketch--S
Adolph-Von-Menzel-The-Flute-Concert-at-Sanssouci-sketch--S 

Agnolo-Bronzino-Venus-Cupid-and-Time-Allegory-of-Lust--S
Agnolo-Bronzino-Venus-Cupid-and-Time-Allegory-of-Lust--S
Agnolo-Bronzino-Venus-Cupid-and-Time-Allegory-of-Lust--S 

Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S+002
Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S+002
Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S+002 

Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S
Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S
Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S 



<strong>NOTE: </strong> Since the os directory list has the 2 values (15 & 16) swapped, let's just swap them for now. 
<br><br>
<strong> BE MINDFUL OF THIS ERROR IN THE FUTURE!!! IT MAY OCCUR FOR DIFFERENT FILES TOO!</strong>

In [157]:
df_asset_folders = pd.DataFrame(os.listdir(to_export));
df_asset_folders.columns = ['folders']
df_asset_folders['folders'][16]

## swapping 15 and 16 to maintain consistency
temp = df_asset_folders['folders'][15]
df_asset_folders['folders'][15] = df_asset_folders['folders'][16]
df_asset_folders['folders'][16] = temp

df_asset_folders['folders'][15]

'Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S+002'

In [158]:
# for special cases:
print(df_originals['genned_set_name'][15])
print (df_genned['copy_to'][15 * 66]) # because the 66th image is the first of the next set for the initial few
print(os.listdir(to_export)[16] ,'\n')

print(df_originals['genned_set_name'][16])
print (df_genned['copy_to'][16 * 66]) # because the 66th image is the first of the next set for the initial few
print(os.listdir(to_export)[15] ,'\n')



Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S+002
Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S+002
Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S+002 

Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S
Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S
Albrecht-Durer-Portrait-of-a-Young-Venetian-Woman-S 



In [169]:
# copying generated files into respective folders

print ('Copying from: ', genned_images)
print ('Copying to: ', to_export, '\n')

path_from = ''
path_to = ''
for i in tqdm(df_genned.values):
    #print(i)
    #print (genned_images)
    #print (to_export)
    path_from = genned_images + i[0]
    path_to = to_export + i[1] + '/' + i[0]
    shutil.copyfile(src = path_from, dst= path_to)

Copying from:  ./processed/
Copying to:  ./Assets_to_import/ 



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1589/1589 [00:18<00:00, 83.95it/s]


### Copy all original-renamed files into respective file directory in buffer_asset folder

In [179]:
# copying original-renamed files into respective folders

print ('Copying from: ', answer_path)
print ('Copying to: ', to_export, '\n')

print('Copying for the first ', len(df_genned['copy_to'].unique()), ' sets.')

path_from = ''
path_to = ''
for i in tqdm(df_originals.values[:len(df_genned['copy_to'].unique())]):
    #print(i)
    #print (answer_path)
    #print (to_export)
    path_from = answer_path + i[2]
    path_to = to_export + i[1] + '/' + i[2]
    shutil.copyfile(src = path_from, dst= path_to)

Copying from:  ./famous-art-512-sq/
Copying to:  ./Assets_to_import/ 

Copying for the first  25  sets.


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25/25 [00:00<00:00, 53.61it/s]


## Generate Json File obj
&nbsp; This is based on our old json structure

In [186]:
#np.ones(len(df_asset_folders))
df_asset_folders
df_asset_folders.columns = ['name']
df_asset_folders['source'] = df_asset_folders['name']
df_asset_folders['answer'] = np.ones(len(df_asset_folders))


ValueError: Length mismatch: Expected axis has 3 elements, new values have 1 elements

In [189]:
df_asset_folders["answer"] = df_asset_folders["answer"].apply(lambda x: str(int(x)))

In [202]:
json_obj = df_asset_folders.transpose()
#json_obj.reset_index()

In [197]:
## saving all processed data so far for easy pickup
df_asset_folders.to_csv('levelDirectory.csv')
df_originals.to_csv('df_originals.csv')
df_genned.to_csv('df_genned.csv')


In [207]:
json_path = to_export + 'levelDirectory.json'
df_asset_folders.to_json(json_path, index = True, orient = 'records')

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
name,Adolph-Von-Menzel-The-Flute-Concert-at-Sanssou...,Agnolo-Bronzino-Venus-Cupid-and-Time-Allegory-...,Albert-Bierstadt-Among-the-Sierra-Nevada-Mount...,Albert-Bierstadt-Deer-in-a-Clearing-S,Albert-Bierstadt-Forest-Sunrise-S,Albert-Bierstadt-Rocky-Mountains-S,Albert-Bierstadt-Rustic-Mill-S,Albert-Bierstadt-Staubbach-Falls-Near-Lauterbr...,Albert-Charles-Lebourg-Albert-Marie-Lebourg-Ro...,Albert-Charles-Lebourg-Notre-Dame-de-Paris-and...,...,Albrecht-Durer-Portrait-of-a-Young-Venetian-Wo...,Albrecht-Durer-Portrait-of-a-Young-Venetian-Wo...,Albrecht-Durer-Self-portrait-at-22-S,Albrecht-Durer-The-Altarpiece-of-the-Rose-Garl...,Albrecht-Durer-The-Martyrdom-of-the-Ten-Thousa...,Albrecht-Durer-Young-Hare-S,AlbrechtDurer-Selfportraitdetailface-S,Alexandre-Cabanel-Fallen-Angel-S,Alexej-Georgewitsch-Von-Jawlensky-Mystical-Hea...,Alfred-Sisley-Foggy-Morning-Voisins-S
source,Adolph-Von-Menzel-The-Flute-Concert-at-Sanssou...,Agnolo-Bronzino-Venus-Cupid-and-Time-Allegory-...,Albert-Bierstadt-Among-the-Sierra-Nevada-Mount...,Albert-Bierstadt-Deer-in-a-Clearing-S,Albert-Bierstadt-Forest-Sunrise-S,Albert-Bierstadt-Rocky-Mountains-S,Albert-Bierstadt-Rustic-Mill-S,Albert-Bierstadt-Staubbach-Falls-Near-Lauterbr...,Albert-Charles-Lebourg-Albert-Marie-Lebourg-Ro...,Albert-Charles-Lebourg-Notre-Dame-de-Paris-and...,...,Albrecht-Durer-Portrait-of-a-Young-Venetian-Wo...,Albrecht-Durer-Portrait-of-a-Young-Venetian-Wo...,Albrecht-Durer-Self-portrait-at-22-S,Albrecht-Durer-The-Altarpiece-of-the-Rose-Garl...,Albrecht-Durer-The-Martyrdom-of-the-Ten-Thousa...,Albrecht-Durer-Young-Hare-S,AlbrechtDurer-Selfportraitdetailface-S,Alexandre-Cabanel-Fallen-Angel-S,Alexej-Georgewitsch-Von-Jawlensky-Mystical-Hea...,Alfred-Sisley-Foggy-Morning-Voisins-S
answer,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


# Ignore the following lines of code; they were made to rename erronous files

In [46]:
'''
to_rename = '.\\to_rename\\';
rename_lst = pd.Series(os.listdir(to_rename));
rename_lst[:5]
'''

"\nto_rename = '.\\to_rename\\';\nrename_lst = pd.Series(os.listdir(to_rename));\nrename_lst[:5]\n"

In [47]:
#rename_to = rename_lst.apply(lambda x: x.split('_')[0] + '--02_' + '_'.join(x.split('_')[1:]))

In [48]:
'''
for i in range(len(rename_lst)):
    filepath = to_rename + "\\" + rename_lst[i];
    filename_to = to_rename + "\\" + rename_to[i]
    os.rename(src = filepath, dst = filename_to)
'''

'\nfor i in range(len(rename_lst)):\n    filepath = to_rename + "\\" + rename_lst[i];\n    filename_to = to_rename + "\\" + rename_to[i]\n    os.rename(src = filepath, dst = filename_to)\n'